In [1]:
import os
os.chdir('..')
%pwd

'd:\\NLP\\Ham_Spam_Classifier-ETE-'

In [16]:
import pandas as pd
from src.entity.artifact_entity import *
from src.utils.s3_operation_utils import S3_operation
from src.utils.utils import *
data = DataTransformationArtifact(
data_transformation_path='artifact\\02_12_2024_16_32_57\\data_transformation\\transform_data\\transform_data.csv',
feature_data_file_path='artifact\\02_12_2024_16_32_57\\data_transformation\\feature_target_data\\Text_pad_sequences.npy', 
label_data_file_path='artifact\\02_12_2024_16_32_57\\data_transformation\\feature_target_data\\label.npy', preprocess_file_path='artifact\\02_12_2024_16_32_57\\data_transformation\\preprocess_file')

model = r"artifact\02_12_2024_02_04_54\model_trainer\model\combined.h5"

In [17]:
X_train, y_train, X_test, y_test = load_and_split_data(data.feature_data_file_path,data.label_data_file_path)

In [24]:
pred = model.predict(X_test[:10])
pred


1/1 [==============================] - 0s 398ms/step


array([[0.1543538 ],
       [0.00453104],
       [0.00687351],
       [0.01788231],
       [0.04779844],
       [0.9999797 ],
       [0.00628578],
       [0.99908817],
       [0.0041835 ],
       [0.00606911]], dtype=float32)

In [27]:
pred_labels = [1 if p >= 0.5 else 0 for p in pred]

In [28]:
pred_labels

[0, 0, 0, 0, 0, 1, 0, 1, 0, 0]

In [29]:
y_test[:10]

array([0, 0, 0, 0, 0, 1, 0, 1, 0, 0], dtype=int64)

In [30]:
ham = "Nah I don't think he goes to usf, he lives around here though"

In [40]:
path = "config\model_info.json"
model_path = read_json(path)
process_model_path = model_path['process_path']
model_path = model_path['model_path']

In [46]:
clean = data_cleaning(ham)

In [45]:
tokenizer = S3_operation().load_process_model(process_model_path)

In [49]:
seq = tokenizer.texts_to_sequences([clean])

In [51]:
padded_seq = pad_sequences(seq, maxlen=60)

In [52]:

padded_seq

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
        765,   7,  26, 342, 711, 178, 133, 343]])

In [54]:
pred = model.predict(padded_seq)
pred

1/1 [==============================] - 0s 86ms/step


array([[0.00191803]], dtype=float32)

In [56]:
pred_labels = [1 if p >= 0.5 else 0 for p in pred]
pred_labels

[0]

In [58]:
model1 = S3_operation().load_model(model_path)

In [61]:
pred = model1.predict(padded_seq)

1/1 [==============================] - 0s 167ms/step


In [62]:
pred_labels = ['spam' if p >= 0.5 else 'ham' for p in pred]
pred_labels

['ham']

In [68]:
df = pd.read_csv(r"research\spam.csv")
spam = df[df.v1=="spam"]['v2'].values

In [70]:
clean = [data_cleaning(i) for i in spam]
clean

['free entri 2 week comp win fa cup final tkts 21st may 2005 text fa 87121 receiv entri questionstd txt ratetc appli 08452810075over18',
 'freemsg hey darl 3 week word back id like fun still tb ok xxx std chgs send 150 rcv',
 'winner valu network custom select receivea 900 prize reward claim call 09061701461 claim code kl341 valid 12 hour',
 'mobil 11 month entitl updat latest colour mobil camera free call mobil updat co free 08002986030',
 'six chanc win cash 100 20000 pound txt csh11 send 87575 cost 150pday 6day 16 tsandc appli repli hl 4 info',
 'urgent 1 week free membership 100000 prize jackpot txt word claim 81010 tc wwwdbuknet lccltd pobox 4403ldnw1a7rw18',
 'xxxmobilemovieclub use credit click wap link next txt messag click httpwap xxxmobilemovieclubcomnqjkgighjjgcbl',
 'england v macedonia dont miss goalsteam news txt nation team 87077 eg england 87077 trywal scotland 4txt120 poboxox36504w45wq 16',
 'thank subscript rington uk mobil charg 5month pleas confirm repli yes repli c

In [79]:
clean[:1]

['free entri 2 week comp win fa cup final tkts 21st may 2005 text fa 87121 receiv entri questionstd txt ratetc appli 08452810075over18']

In [82]:
seq = tokenizer.texts_to_sequences(clean[:1])
seq

[[14,
  416,
  3,
  32,
  808,
  135,
  2199,
  1110,
  308,
  2200,
  2201,
  207,
  2202,
  21,
  2199,
  2203,
  213,
  416,
  3261,
  38,
  3262,
  321,
  3263]]

In [89]:
def preprocess(message):
    clean = data_cleaning(message)
    seq = tokenizer.texts_to_sequences([clean])
    padded_seq = pad_sequences(seq, maxlen=60)
    y_pred = model.predict(padded_seq)
    y_pred_prob = ['spam' if y_pred>0.5 else 'ham' for i in y_pred]

    return y_pred_prob,y_pred,message



In [91]:
data = "WINNER!! As a valued network customer you have been selected to receivea �900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only."
preprocess(data)

1/1 [==============================] - 0s 69ms/step


(['spam'], array([[0.9959394]], dtype=float32))

In [84]:
padded_seq

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,   14,  416,    3,   32,  808,  135, 2199,
        1110,  308, 2200, 2201,  207, 2202,   21, 2199, 2203,  213,  416,
        3261,   38, 3262,  321, 3263]])

In [85]:
pred = model1.predict(padded_seq)

1/1 [==============================] - 0s 83ms/step


In [86]:
pred

array([[0.99709135]], dtype=float32)

In [ ]:
pred_y = [for i in pred]

In [ ]:
path = "config\model_info.json"
model_path = read_json(path)
process_model_path = model_path['process_path']
model_path = model_path['model_path']

{'process_path': 'preprocess_model/preprocess.pkl',
 'model_path': 'New_version_model/rnn.h5'}

In [ ]:
process_model_path = model_path['process_path']
model_path = model_path['model_path']

In [ ]:
tokenizer = S3_operation().load_process_model(process_model_path)
model = S3_operation().load_model(model_path)
tokenizer.word_index

{'call': 1,
 'im': 2,
 '2': 3,
 'go': 4,
 'get': 5,
 'come': 6,
 'dont': 7,
 '4': 8,
 'know': 9,
 'ltgt': 10,
 'ok': 11,
 'like': 12,
 'got': 13,
 'free': 14,
 'want': 15,
 'time': 16,
 'love': 17,
 'good': 18,
 'day': 19,
 'ill': 20,
 'text': 21,
 'need': 22,
 'send': 23,
 'one': 24,
 'see': 25,
 'think': 26,
 'youknow': 27,
 'home': 28,
 'take': 29,
 'today': 30,
 'back': 31,
 'week': 32,
 'still': 33,
 'tell': 34,
 'stop': 35,
 'repli': 36,
 'make': 37,
 'txt': 38,
 'well': 39,
 'say': 40,
 'mobil': 41,
 'buddi': 42,
 'ask': 43,
 'n': 44,
 'new': 45,
 'hi': 46,
 'phone': 47,
 'pleas': 48,
 'sorri': 49,
 'work': 50,
 'hope': 51,
 'oh': 52,
 'meet': 53,
 'miss': 54,
 'hey': 55,
 'night': 56,
 'wait': 57,
 'thing': 58,
 'cant': 59,
 'give': 60,
 'much': 61,
 'tri': 62,
 'way': 63,
 'na': 64,
 'great': 65,
 'wat': 66,
 'thank': 67,
 'happi': 68,
 'later': 69,
 'number': 70,
 'claim': 71,
 'that': 72,
 'dear': 73,
 'laugh': 74,
 'wan': 75,
 'messag': 76,
 'alreadi': 77,
 'friend': 78,
 '

In [ ]:
tokenizer.word_index

{'call': 1,
 'im': 2,
 '2': 3,
 'go': 4,
 'get': 5,
 'come': 6,
 'dont': 7,
 '4': 8,
 'know': 9,
 'ltgt': 10,
 'ok': 11,
 'like': 12,
 'got': 13,
 'free': 14,
 'want': 15,
 'time': 16,
 'love': 17,
 'good': 18,
 'day': 19,
 'ill': 20,
 'text': 21,
 'need': 22,
 'send': 23,
 'one': 24,
 'see': 25,
 'think': 26,
 'youknow': 27,
 'home': 28,
 'take': 29,
 'today': 30,
 'back': 31,
 'week': 32,
 'still': 33,
 'tell': 34,
 'stop': 35,
 'repli': 36,
 'make': 37,
 'txt': 38,
 'well': 39,
 'say': 40,
 'mobil': 41,
 'buddi': 42,
 'ask': 43,
 'n': 44,
 'new': 45,
 'hi': 46,
 'phone': 47,
 'pleas': 48,
 'sorri': 49,
 'work': 50,
 'hope': 51,
 'oh': 52,
 'meet': 53,
 'miss': 54,
 'hey': 55,
 'night': 56,
 'wait': 57,
 'thing': 58,
 'cant': 59,
 'give': 60,
 'much': 61,
 'tri': 62,
 'way': 63,
 'na': 64,
 'great': 65,
 'wat': 66,
 'thank': 67,
 'happi': 68,
 'later': 69,
 'number': 70,
 'claim': 71,
 'that': 72,
 'dear': 73,
 'laugh': 74,
 'wan': 75,
 'messag': 76,
 'alreadi': 77,
 'friend': 78,
 '

In [ ]:
ham = "Ok lar... Joking wif u oni..."
spam = "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

In [ ]:
from src.utils.utils import data_cleaning
clean_data = data_cleaning(ham)

clean_data

'ok lar joke oni'

In [ ]:
sequences = tokenizer.texts_to_sequences([clean_data])
sequences

[[], [114], [], [], [], [], [], [1222], [], [114], [88], [], [], [44], []]

In [ ]:
padded_seq = pad_sequences(sequences, maxlen=60,padding='post')
padded_seq

array([[  11,  226,  579, 1448,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0]])

In [ ]:
prediction = model.predict(padded_seq)
label = 'ham' if prediction[0][0] > 0.5 else 'spam'

1/1 [==============================] - 1s 1s/step


In [ ]:
label

'spam'

In [ ]:
def model_pred(data):
    clean_data = data_cleaning(data)
    vector = tokenizer